# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!Powershell.exe -Command type publications.tsv -Head 5

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2023-05-15	Bacteria disinfection of rice seeds by ultraviolet light irradiation in a biosafe flow cabinet	Plant Health Progress		"Jia, Y. and Q. D. Read. 2023. Bacteria disinfection of rice seeds by ultraviolet light irradiation in a biosafe flow cabinet. Plant Health Progress. DOI: 10.1094/PHP-02-23-0017-RS."	jia-and-read-2023	https://doi.org/10.1094/PHP-02-23-0017-RS
2023-04-11	Optimal timing of fungicide application to manage Fusarium head blight in winter barley	Plant Disease		"Cowger, C., Q. D. Read, L. Clark, and Y. Dong. 2023. Optimal timing of fungicide application to manage Fusarium head blight in winter barley. Plant Disease. DOI: 10.1094/PDIS-01-23-0021-RE."	cowger-et-al-2023	https://doi.org/10.1094/PDIS-01-23-0021-RE
2023-03-16	Evaluating the accuracy and efficiency of test weight instruments for soybean (Glycine max L.) research	Agrosystems, Geosciences & Environment	Open access!	"Fett, R., A. M. Gillen, Q. D. Read, 

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications.head()


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2023-06-12,Use of automated capillary immunoassay for qua...,Journal of Microbiological Methods,NaN,"Yeh, H.-Y., J. G. Frye, C. R. Jackson, Q. D. R...",yeh-et-al-2023,https://doi.org/10.1016/j.mimet.2023.106757
1,2023-05-19,Parental choice and seed size impact the uprig...,Crop Science,NaN,"Taliercio, E., D. Eickholt, Q. D. Read, T. Car...",taliercio-et-al-2023,https://doi.org/10.1002/csc2.21015
2,2023-05-15,Bacteria disinfection of rice seeds by ultravi...,Plant Health Progress,NaN,"Jia, Y. and Q. D. Read. 2023. Bacteria disinfe...",jia-and-read-2023,https://doi.org/10.1094/PHP-02-23-0017-RS
3,2023-04-11,Optimal timing of fungicide application to man...,Plant Disease,NaN,"Cowger, C., Q. D. Read, L. Clark, and Y. Dong....",cowger-et-al-2023,https://doi.org/10.1094/PDIS-01-23-0021-RE
4,2023-03-16,Evaluating the accuracy and efficiency of test...,"Agrosystems, Geosciences & Environment",Open access!,"Fett, R., A. M. Gillen, Q. D. Read, S. Patel, ...",fett-et-al-2023,https://doi.org/10.1002/agg2.20354


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [4]:
# Here, subset the rows of publications to only 1 or however many we need to update right now
publications = publications.head(1)

In [5]:
# Some test code to see what the filename would be
str(publications['pub_date'])

'0    2023-06-12\n1    2023-05-19\nName: pub_date, dtype: object'

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    ##md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [7]:
%%cmd
dir ..\_publications\

Microsoft Windows [Version 10.0.19044.2965]
(c) Microsoft Corporation. All rights reserved.

C:\Users\qdread\Documents\GitHub\qdread.github.io\markdown_generator>dir ..\_publications\
 Volume in drive C has no label.
 Volume Serial Number is 08D0-A2BB

 Directory of C:\Users\qdread\Documents\GitHub\qdread.github.io\_publications

06/14/2023  08:56 PM    <DIR>          .
06/14/2023  08:56 PM    <DIR>          ..
01/19/2023  10:58 AM               724 2012-02-29-clark-et-al-2012.md
01/19/2023  10:58 AM               708 2013-11-12-vannuland-et-al-2013.md
01/19/2023  10:58 AM               776 2013-12-07-gorman-et-al-2013.md
01/19/2023  10:58 AM               871 2014-02-01-read-et-al-2014-functional-ecology.md
01/19/2023  10:58 AM               741 2015-09-01-schussler-et-al-2015.md
01/19/2023  10:58 AM               806 2016-07-01-read-et-al-2016-oikos.md
01/19/2023  10:58 AM               682 2016-07-01-vannuland-et-al-2016.md
01/19/2023  10:58 AM               773 2016-08-01-yoon-and-